# Klassifikation 

In diesem Notebook trainieren wir einen Classifier. Aus Beispielen lernt der Classifier, zu welcher Klasse ein Element gehört, und kann neue Elemente mit den gelernten Regeln einordnen. 

Wir lesen zunächst Dokumente aus 4 Wikipedia-Kategorien ein.

### Texten sammeln (API)

In [1]:
import SaveWiki

SaveWiki.downloadWikiCat('Automobil','Automobil')
SaveWiki.downloadWikiCat('Elektroauto','Elektroauto')

##### Texten aufmachen und lesen

In [3]:
import codecs 

def readtext(dateiname):
    text = ''
    d = codecs.open(dateiname,'r','utf8')
    for zeile in d:
        text += str(zeile)
    d.close()

    return text

In [4]:
bsptext= readtext('krebs/Adenokarzinom.txt')
print(bsptext[:300])

Als Adenokarzinom bezeichnet man einen bösartigen (malignen) von der Deckzellschicht (Epithel) ausgehenden Tumor, der aus Drüsengewebe hervorgegangen ist. Die gutartige (benigne) Zellveränderung von Drüsengewebe nennt man dagegen Adenom.
Adenokarzinome kommen vor allem im Bereich der Verdauungsorgan


### Merkmalen des Textes auswählen

***stopwords ausschließen***

***Zeichen ausschließen***

***closed class words ausschließen***

In [5]:
#stopwords

In [6]:
import nltk
import re
from HanTa import HanoverTagger as ht
from collections import Counter

stopwords = nltk.corpus.stopwords.words('german')
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

def closed_class(pos):
    if pos[0] == '$':
        return True
    elif pos in ["APPR", "APPRART", "APPO", "APZR", "ART", "KOUI", "KOUS", "KON", "KOKOM", "PDS", "PDAT", "PIS", "PIAT", "PIDAT", "PPER", "PPOSS", "PPOSAT", "PRELS", "PRELAT", "PRF", "PWS", "PWAT", "PWAV", "PAV", "PTKZU", "PTKNEG", "VAFIN", "VAIMP", "VAINF", "VAPP", "VMFIN", "VMINF", "VMPP"]:
        return True
    
    return False

def features_from_text(text):
    wordcounts = Counter()
    tlen = 0
    
    satzliste =  nltk.sent_tokenize(text,language='german')
    for satz in satzliste:
        tokens =  nltk.word_tokenize(satz,language='german')
        tokens = [lemma for (word,lemma,pos) in tagger.tag_sent(tokens) if not closed_class(pos)]
        tokens = [t for t in tokens if t.lower() not in stopwords]
        tokens = [t for t in tokens if re.search('^\w+$',t)]
        tlen += len(tokens)
        wordcounts.update(tokens)

    return {w:wordcounts[w]/tlen for w in wordcounts}

Wir versuchen aus jede Klasse 50 Dokumente zu lesen, die nicht extrem kurz oder lang sind.

In [8]:
import glob

def read_data(directories):
    docs = []
    for directory in directories:
        dirsize = 0
        for file in glob.glob(directory+"/*.txt"):
            text = readtext(file)
            if len(text) > 500 and len(text) < 10000:
                docs.append((features_from_text(text),directory))
                dirsize += 1
            if dirsize >= 50:
                break
    return docs

#data = read_data(['infekt','krebs','pneumologie','kardiologie'])


data = read_data(['Automobil','Elektroauto'])

In [9]:
data[1]

({'66': 0.05056179775280899,
  'denkmalgeschützt': 0.0056179775280898875,
  'Autokino': 0.03932584269662921,
  'Carthage': 0.011235955056179775,
  'Missouri': 0.0056179775280898875,
  'Jahr': 0.02247191011235955,
  '2003': 0.011235955056179775,
  'National': 0.011235955056179775,
  'Register': 0.011235955056179775,
  'of': 0.0056179775280898875,
  'historic': 0.0056179775280898875,
  'Place': 0.011235955056179775,
  'Nrhp': 0.0056179775280898875,
  'offiziell': 0.011235955056179775,
  'Kulturdenkmallist': 0.011235955056179775,
  'Bundesregierung': 0.0056179775280898875,
  'aufnehmen': 0.011235955056179775,
  'Geschichte': 0.0056179775280898875,
  'original': 0.0056179775280898875,
  'kurz': 0.0056179775280898875,
  'Fertigstellung': 0.011235955056179775,
  'September': 0.016853932584269662,
  '1949': 0.016853932584269662,
  'Route': 0.0056179775280898875,
  'Rand': 0.0056179775280898875,
  'Stadt': 0.0056179775280898875,
  'eröffnen': 0.016853932584269662,
  'Bauarbeit': 0.005617977528

Wir mischen die Daten und teilen in Test- und Trainingsdaten

In [10]:
len(data)

60

In [11]:
import random

random.shuffle(data)
train_data = data[:40]
test_data = data[40:]

Wir schauen uns jetzt mal ein Dokument an:

In [12]:
len(train_data[27])

2

Wir haben jetzt für jedes Dokument einene Merkmalsvektor. Man merke: der Vektor ist eigentlich so lang wie die Anzahl der unterschiedlichen Wörter in der ganzen Sammlung. Alle Wörter, die nicht erwähnt werden haben den Wert 0. 

Wörter, die nur in ein oder zwei Dokumementen vorkommen sind, für die Klassifikation nicht besonders nützlich. 
Wir nutzen nachher nur die Wörter, die in mindestens 5 Dokumente vorkommen. Um das vorzubereiten, berechnen wir für alle Wörter die Dokumentfrequenz>

In [13]:
docfreq = Counter()
for (wfreq,c) in train_data:
    docfreq.update(wfreq.keys())

In [14]:
docfreq

Counter({'Kolbenfresser': 1,
         'Österreich': 4,
         'Kolbenreiber': 1,
         'Kolbenanreiber': 1,
         'Kolbenverreiber': 1,
         'Hubkolbenmotor': 1,
         'Schaden': 1,
         'bewegt': 2,
         'Kolbe': 2,
         'Bereich': 8,
         'Berührungsfläche': 1,
         'Zylinder': 2,
         'bezeichnen': 13,
         'Ursache': 1,
         'mangelhaft': 1,
         'Schmierung': 1,
         'Lauffläche': 1,
         'entstehen': 8,
         'trocken': 1,
         'Reibung': 1,
         'Temperatur': 1,
         'stark': 5,
         'erhöhen': 3,
         'dadurch': 5,
         'Reibschweißung': 1,
         'einleiten': 1,
         'Zylinderlaufbuchs': 1,
         'verbinden': 2,
         'fest': 3,
         'miteinander': 1,
         'Maschine': 1,
         'schlagartig': 1,
         'Stillstand': 1,
         'bringen': 6,
         'dafür': 3,
         'erstrecken': 1,
         'unterschiedlich': 6,
         'Grund': 6,
         'erhöht': 1,
        

## Klassifikation mit Scikit Learn

Die Bibliothek Scikit Learn stellt verschiedene Klassifikationsmodelle zur Verfügung. Wir müssen jetzt richtige Merkmalsvektoren aufbauen, bei denen die Position die Bedeutung einer Zahl bestimmt. Dazu machen wir erst eine feste Liste mit allen Wörtern. Wörter, die zu selten sind lassen wir weg.

In [15]:
from sklearn import linear_model, datasets


allfeatures = [w for w in docfreq if docfreq[w] > 4]

def make_feat_vec(featmap,featlist):
    vec = []
    for f in featlist:
        vec.append(featmap.get(f,0.0))
    return vec

train_vec =  [make_feat_vec(feats,allfeatures) for feats,cls in train_data]
train_label = [cls for feats,cls in train_data]

Wir schauen uns mal einen Vektor an:

In [16]:
len(train_vec[26])

178

In [17]:
train_label[26]

'Automobil'

Ein einfaches, aber effektives Klassifikationsmodell, das meistens gute Ergebnisse liefert, ist die logistische Regression.

In [18]:
logreg = linear_model.LogisticRegression(C=1e9,verbose=True)
logreg.fit(train_vec,train_label)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


LogisticRegression(C=1000000000.0, verbose=True)

Wir können jetzt mit dem trainierten Classifier ein neues Dokument klassifizieren:

In [19]:
v = make_feat_vec(test_data[17][0],allfeatures) 
logreg.predict([v])

array(['Automobil'], dtype='<U11')

Richtig ist:

In [20]:
test_data[17][1]

'Automobil'

Oder direkt für alles Testdaten:

In [21]:
test_vec = [make_feat_vec(feats,allfeatures) for feats,cls in test_data]
test_label = [cls for feats,cls in test_data]

pred_label = list(logreg.predict(test_vec))

In [22]:
correct = 0
for i in range(len(test_label)):
    if test_label[i] == pred_label[i]:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/len(test_label)))

60.0 Prozent korrekt


### Evaluation

f-Faktor
Recall
Precision
Stearman

Wo sind aber die Fehler aufgetreten? Un diese Frage zu beantworten, ist sogenannte eine _Konfusionsmatrix_ sehr hilfreich. NLTK hat eine Funktion zum erstellen eines Konfusionsmatrizen:

In [23]:
cm = nltk.ConfusionMatrix(test_label, pred_label)
print(cm)

            |     E |
            |     l |
            |  A  e |
            |  u  k |
            |  t  t |
            |  o  r |
            |  m  o |
            |  o  a |
            |  b  u |
            |  i  t |
            |  l  o |
------------+-------+
  Automobil |<12> 1 |
Elektroauto |  7 <.>|
------------+-------+
(row = reference; col = test)



Es gibt verschiedene andere Klassifikatorn, die wie nutzen können:

In [23]:
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier(n_neighbors = 3)
knn.fit(train_vec,train_label)
pred_label = list(knn.predict(test_vec))
correct = 0
for i in range(len(test_label)):
    if test_label[i] == pred_label[i]:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/len(test_label)))

65.0 Prozent korrekt


In [24]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(train_vec,train_label)
pred_label = list(rf.predict(test_vec))
correct = 0
for i in range(len(test_label)):
    if test_label[i] == pred_label[i]:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/len(test_label)))

82.5 Prozent korrekt
